<a href="https://colab.research.google.com/github/RaminTKhameneh/NCMBCP/blob/main/IWO_COR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Colab**

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# **Libraries**

In [ ]:
import numpy as np
from copy import deepcopy as dc
from time import process_time
import random
from math import exp, floor

# **Read the Data**

In [ ]:
def Read_location(file):
    f = open('/content/drive/MyDrive/Colab Notebooks/MBCP/'+file, 'r')
    data = []
    for line in f:
        data.append([float(v) for v in line.split()])
    f.close()
    return data

def Read_donation(file):
    f = open('/content/drive/MyDrive/Colab Notebooks/MBCP/'+file, 'r')
    data = []
    for line in f:
        data.append([float(v) for v in line.split()])
    f.close()
    return data

# **Main Functions**

Common functions

In [ ]:
def duration(rout,total_distance):
    dur = 0
    for i in range(len(rout) -1) :
        dur+= total_distance[rout[i]][rout[i+1]]
    return dur

def degree_n_poss(old_poss,first_list,total_distance):
    new_poss = []
    for i in old_poss :
        for j in first_list :
            if duration (i+[j,0],total_distance) <= spt and j not in i:
                new_poss.append(i+[j])
    return new_poss

def minTest(roots,total_distance):
    minimumTest = []
    for i in roots:
        minimumTest.append(duration(i,total_distance))
    minimum = min(minimumTest)
    return minimum

def meetTime(rout,start_time,total_distance):
    meet_time = [start_time]
    for i in range(len(rout)-1):
        meet_time.append(start_time + duration([rout[i],rout[i+1]],total_distance))
        start_time = meet_time[-1]
    return meet_time

def multi_healthy_collected_plt (rout,start_time,donation_data,spt,total_distance):
    don_data = dc(donation_data)
    pick_list = []
    for i in range(len(rout)):
        meet_time = meetTime(rout[i],start_time[i],total_distance)
        pickup = []
        for j in range(len(rout[i])):
            p = 0
            for k in range(len(don_data[rout[i][j]])):
                if don_data[rout[i][j]][k] <= meet_time[j] and don_data[rout[i][j]][k] >= meet_time[-1]-spt:
                    p += 1
                    don_data[rout[i][j]][k] = 99999
            pickup.append(p)
        pick_list.append(sum(pickup))
    return sum(pick_list)

def random_start (remain,multipleRout,total_distance):
    size = len(multipleRout) + 1
    temp = np.random.dirichlet(np.ones(size),size=1)[0]
    finalRemain = [np.floor(num) for num in (temp * (remain+1))]
    finalRemain.pop()
    for i in range(len(finalRemain)-1):
        finalRemain[i+1] += finalRemain[i]
    times = [0]
    for rout in multipleRout:
        times.append(meetTime(rout,times[-1],total_distance)[-1])
    times.pop()
    for i in range(len(times)):
        times[i] = times[i] + finalRemain[i]
    return times

def improved_st (initial_sol,initial_time,donation_data,spt,total_distance):
    solution = dc(initial_sol)
    start_time = dc(initial_time)
    rem_time = work_hours - sum([duration(i,total_distance) for i in solution])
    if rem_time < 0 :
        return initial_time
    else:
        for i in range(15):
            temp_start_time = random_start (rem_time,solution,total_distance)
            if multi_healthy_collected_plt(solution,start_time,donation_data,spt,total_distance) < multi_healthy_collected_plt(solution,temp_start_time,donation_data,spt,total_distance):
                start_time = dc(temp_start_time)
        return start_time

def initial_timing (sol,total_distance):
    initial_time = ([duration(i,total_distance) for i in sol])
    for i in range(len(initial_time)-1) :
        initial_time[i+1] = initial_time[i]+initial_time[i+1]
    initial_time = [0]+initial_time
    initial_time.pop()
    return initial_time

def updating_donation_list (rout,start_time,donation_data,total_distance):
    don_data = dc(donation_data)
    global spt
    for i in range(len(rout)):
        meet_time = meetTime(rout[i],start_time[i],total_distance)
        for j in range(len(rout[i])):
            for k in range(len(don_data[rout[i][j]])):
                if don_data[rout[i][j]][k] <= meet_time[j] and don_data[rout[i][j]][k] >= meet_time[-1]-spt:
                    don_data[rout[i][j]][k] = 99999
    return don_data



Creating Initial Population

In [ ]:
def initial_population(population_size,roots,min_test,work_hours,donation_data,spt,total_distance):
    population = []
    population_timing = []
    for i in range(population_size):
        multiSeq = []
        multiTime = [0]
        a = dc(random.choice (roots)) 
        multiTime.append(duration(a,total_distance)+multiTime[-1])
        multiSeq.append(a)
        while  multiTime[-1] + min_test < work_hours :
            a = dc(random.choice (roots))
            multiSeq.append(a)
            multiTime.append(multiTime[-1]+duration(a,total_distance))
        multiTime = multiTime[:-2]
        multiSeq.pop()
        population.append(multiSeq)
        population_timing.append(improved_st (multiSeq,multiTime,donation_data,spt,total_distance))
    return population,population_timing

def chrom_pool (donation_data,spt,the_size,total_distance):
    x = list(range(1,len(donation_data)))
    first_list = []
    for i in x:
        if duration([i,0],total_distance) <= spt :
            first_list.append(i)
    poss_one = [[i]for i in first_list]
    old_poss = poss_one
    final_list = poss_one
    while len(final_list) < the_size and len(old_poss) > 0 :
        old_poss = degree_n_poss(old_poss,first_list,total_distance)
        final_list.extend(old_poss)      
    routs =final_list
    roots = [[0]+i+[0] for i in routs]
    return roots

# **verification**

In [ ]:
def counting_check (rout,start_time,donation_data,total_distance,spt,best_result):
    don_data = dc(donation_data)
    for z in range(len(rout)):
        for i in range(len(rout[z])):
            meet_time = meetTime(rout[z][i],start_time[z][i],total_distance)
            for j in range(len(rout[z][i])):
                for k in range(len(don_data[rout[z][i][j]])):
                    if don_data[rout[z][i][j]][k] <= meet_time[j] and don_data[rout[z][i][j]][k] >= meet_time[-1]-spt:
                        don_data[rout[z][i][j]][k] = 88888
    counter  = 0
    for row in don_data :
        for item in row :
            if item == 88888 :
                counter +=1
    if counter != sum(best_result):
        print('there is a conflict')

def CP_check(donation_data,best_result):
    counter = 0
    for i in donation_data:
        for j in i :
            if j == 99999:
                counter+=1
    if counter != sum(best_result):
        ('there is a conflict in CP counting')
        
def time_conflict(routes,timing,total_distance):
    for i in range(len(routes)):
        flag = 0
        for j in range(len(routes[i])-1):
            if timing[i][j]+duration(routes[i][j],total_distance)> timing [i][j+1]:
            #     pass
            # else:
                flag = 1
                break
        if flag == 1 :
            print('there is a conflict between timings for vehicle: ', i+1)
            print(timing[i])
            print(routes[i])
            break

In [ ]:
def two_opt(tour_org,total_distance,spt, iter, MaxIter, sigma_initial, sigma_final, roots):
    tour = dc(tour_org)
    if  random.uniform(0,1) > 0.85: ############## 0.4 ==> 0.7 ==>
        opt1 = random.randint(0,len(tour)-1)
        opt2 = random.randint(0,len(tour)-1)
        tour[opt1],tour[opt2] = tour[opt2],tour[opt1]
    else:
        opt1 = random.randint(0,len(tour)-1)
        new_route = dc(random.choice (roots))
        tour[opt1] = new_route
    timing = initial_timing (tour,total_distance)
    timing = spatial_st (tour,timing,donation_data,spt,total_distance, iter, MaxIter, sigma_initial,sigma_final)
    return tour, timing


def weak_del (fitness,population,population_timing,p_min):
    for i in range(len(fitness)):
        if len(fitness) <= p_min:
            break
        weak_id = dc(np.argmin(fitness))
        population.pop(weak_id)
        population_timing.pop(weak_id)
        fitness.pop(weak_id)
    return fitness,population,population_timing



**spatial**

In [ ]:
def spatial_st (initial_sol,initial_time,donation_data,spt,total_distance,iter, MaxIter, sigma_initial,sigma_final):
    solution = dc(initial_sol)
    start_time = dc(initial_time)
    rem_time = work_hours - sum([duration(i,total_distance) for i in solution])
    if rem_time < 0 :
        return initial_time
    else:
        spatial_range = dc(spatial(iter, MaxIter, sigma_initial,sigma_final))
        for i in range(spatial_range):
            temp_start_time = random_start (rem_time,solution,total_distance)
            if multi_healthy_collected_plt(solution,start_time,donation_data,spt,total_distance) < multi_healthy_collected_plt(solution,temp_start_time,donation_data,spt,total_distance):
                start_time = dc(temp_start_time)
        return start_time


def spatial(iter, MaxIter, sigma_initial,sigma_final):
    sigma = (((MaxIter - iter)/(MaxIter - 1))**2) * (sigma_initial - sigma_final) + sigma_final;
    return (int(sigma*15)+1)


# **Problem Configuration**

In [ ]:

dons = ('don1.txt','don2.txt','don3.txt','don4.txt','don5.txt','don6.1.txt','don7.txt')

locs = ('loc1.1.txt','loc2.1.txt','loc3.1.txt','loc4.1.txt','loc5.1.txt','loc6.1.txt','loc7.1.txt')

vehicle_num = (6,6,7,8,9,10,10)
spoil_time = (300,500,700)

sigma_initial = 1
sigma_final = 0.01
MaxIter = 25      ###################### 30  ==> 25



In [ ]:
for i in range(7):
    print(len(Read_location(locs[i])))
    # print(Read_location(locs[i]))
    # print()


28
32
36
38
43
46
51


In [ ]:
a = [[0.0, 380.0, 410.0, 309.0, 403.0, 425.0, 304.0, 367.0, 500.0, 331.0, 408.0, 430.0, 314.0, 314.0, 117.0, 171.0, 216.0, 220.0, 131.0, 299.0, 114.0, 277.0, 103.0, 108.0, 358.0, 354.0, 400.0, 347.0], [380.0, 0.0, 32.0, 248.0, 377.0, 381.0, 199.0, 421.0, 241.0, 253.0, 44.0, 160.0, 137.0, 377.0, 262.0, 234.0, 209.0, 400.0, 445.0, 436.0, 370.0, 151.0, 464.0, 327.0, 452.0, 31.0, 324.0, 119.0], [410.0, 32.0, 0.0, 258.0, 380.0, 382.0, 209.0, 429.0, 222.0, 260.0, 23.0, 147.0, 149.0, 389.0, 292.0, 266.0, 241.0, 419.0, 477.0, 450.0, 402.0, 169.0, 492.0, 359.0, 462.0, 56.0, 325.0, 125.0], [309.0, 248.0, 258.0, 0.0, 139.0, 151.0, 49.0, 171.0, 210.0, 22.0, 239.0, 170.0, 111.0, 131.0, 235.0, 283.0, 300.0, 190.0, 429.0, 198.0, 375.0, 110.0, 340.0, 334.0, 204.0, 219.0, 104.0, 133.0], [403.0, 377.0, 380.0, 139.0, 0.0, 24.0, 185.0, 82.0, 250.0, 127.0, 359.0, 256.0, 245.0, 100.0, 357.0, 415.0, 436.0, 212.0, 533.0, 165.0, 489.0, 250.0, 401.0, 453.0, 126.0, 351.0, 59.0, 257.0], [425.0, 381.0, 382.0, 151.0, 24.0, 0.0, 195.0, 103.0, 237.0, 136.0, 360.0, 252.0, 252.0, 124.0, 376.0, 431.0, 451.0, 236.0, 554.0, 188.0, 510.0, 260.0, 426.0, 473.0, 146.0, 356.0, 57.0, 262.0], [304.0, 199.0, 209.0, 49.0, 185.0, 195.0, 0.0, 221.0, 196.0, 59.0, 192.0, 139.0, 62.0, 178.0, 215.0, 252.0, 263.0, 223.0, 417.0, 242.0, 357.0, 64.0, 350.0, 314.0, 252.0, 170.0, 142.0, 86.0], [367.0, 421.0, 429.0, 171.0, 82.0, 103.0, 221.0, 0.0, 327.0, 169.0, 410.0, 321.0, 284.0, 56.0, 344.0, 412.0, 441.0, 154.0, 498.0, 89.0, 465.0, 277.0, 347.0, 434.0, 43.0, 391.0, 134.0, 304.0], [500.0, 241.0, 222.0, 210.0, 250.0, 237.0, 196.0, 327.0, 0.0, 193.0, 200.0, 84.0, 192.0, 313.0, 401.0, 416.0, 411.0, 398.0, 605.0, 389.0, 539.0, 227.0, 546.0, 494.0, 369.0, 233.0, 191.0, 167.0], [331.0, 253.0, 260.0, 22.0, 127.0, 136.0, 59.0, 169.0, 193.0, 0.0, 241.0, 160.0, 118.0, 135.0, 256.0, 302.0, 318.0, 205.0, 452.0, 206.0, 397.0, 123.0, 361.0, 356.0, 205.0, 225.0, 85.0, 135.0], [408.0, 44.0, 23.0, 239.0, 359.0, 360.0, 192.0, 410.0, 200.0, 241.0, 0.0, 124.0, 132.0, 370.0, 292.0, 271.0, 250.0, 405.0, 481.0, 434.0, 407.0, 157.0, 486.0, 363.0, 444.0, 56.0, 303.0, 106.0], [430.0, 160.0, 147.0, 170.0, 256.0, 252.0, 139.0, 321.0, 84.0, 160.0, 124.0, 0.0, 116.0, 294.0, 324.0, 334.0, 327.0, 359.0, 529.0, 367.0, 460.0, 152.0, 486.0, 415.0, 361.0, 149.0, 197.0, 85.0], [314.0, 137.0, 149.0, 111.0, 245.0, 252.0, 62.0, 284.0, 192.0, 118.0, 132.0, 116.0, 0.0, 240.0, 209.0, 225.0, 227.0, 273.0, 413.0, 301.0, 346.0, 36.0, 376.0, 301.0, 314.0, 107.0, 197.0, 34.0], [314.0, 377.0, 389.0, 131.0, 100.0, 124.0, 178.0, 56.0, 313.0, 135.0, 370.0, 294.0, 240.0, 0.0, 288.0, 356.0, 386.0, 111.0, 446.0, 75.0, 410.0, 229.0, 303.0, 378.0, 74.0, 347.0, 131.0, 264.0], [117.0, 262.0, 292.0, 235.0, 357.0, 376.0, 215.0, 344.0, 401.0, 256.0, 292.0, 324.0, 209.0, 288.0, 0.0, 78.0, 120.0, 231.0, 204.0, 302.0, 141.0, 172.0, 208.0, 99.0, 350.0, 236.0, 338.0, 239.0], [171.0, 234.0, 266.0, 283.0, 415.0, 431.0, 252.0, 412.0, 416.0, 302.0, 271.0, 334.0, 225.0, 356.0, 78.0, 0.0, 44.0, 308.0, 210.0, 377.0, 136.0, 194.0, 273.0, 93.0, 422.0, 215.0, 387.0, 248.0], [216.0, 209.0, 241.0, 300.0, 436.0, 451.0, 263.0, 441.0, 411.0, 318.0, 250.0, 327.0, 227.0, 386.0, 120.0, 44.0, 0.0, 346.0, 243.0, 412.0, 168.0, 201.0, 318.0, 129.0, 454.0, 195.0, 404.0, 245.0], [220.0, 400.0, 419.0, 190.0, 212.0, 236.0, 223.0, 154.0, 398.0, 205.0, 405.0, 359.0, 273.0, 111.0, 231.0, 308.0, 346.0, 0.0, 351.0, 78.0, 328.0, 248.0, 193.0, 304.0, 137.0, 368.0, 235.0, 305.0], [131.0, 445.0, 477.0, 429.0, 533.0, 554.0, 417.0, 498.0, 605.0, 452.0, 481.0, 529.0, 413.0, 446.0, 204.0, 210.0, 243.0, 351.0, 0.0, 429.0, 75.0, 377.0, 192.0, 118.0, 488.0, 425.0, 526.0, 443.0], [299.0, 436.0, 450.0, 198.0, 165.0, 188.0, 242.0, 89.0, 389.0, 206.0, 434.0, 367.0, 301.0, 75.0, 302.0, 377.0, 412.0, 78.0, 429.0, 0.0, 406.0, 284.0, 264.0, 380.0, 61.0, 405.0, 205.0, 329.0], [114.0, 370.0, 402.0, 375.0, 489.0, 510.0, 357.0, 465.0, 539.0, 397.0, 407.0, 460.0, 346.0, 410.0, 141.0, 136.0, 168.0, 328.0, 75.0, 406.0, 0.0, 311.0, 207.0, 44.0, 462.0, 351.0, 476.0, 374.0], [277.0, 151.0, 169.0, 110.0, 250.0, 260.0, 64.0, 277.0, 227.0, 123.0, 157.0, 152.0, 36.0, 229.0, 172.0, 194.0, 201.0, 248.0, 377.0, 284.0, 311.0, 0.0, 340.0, 266.0, 304.0, 120.0, 207.0, 70.0], [103.0, 464.0, 492.0, 340.0, 401.0, 426.0, 350.0, 347.0, 546.0, 361.0, 486.0, 486.0, 376.0, 303.0, 208.0, 273.0, 318.0, 193.0, 192.0, 264.0, 207.0, 340.0, 0.0, 210.0, 325.0, 435.0, 414.0, 410.0], [108.0, 327.0, 359.0, 334.0, 453.0, 473.0, 314.0, 434.0, 494.0, 356.0, 363.0, 415.0, 301.0, 378.0, 99.0, 93.0, 129.0, 304.0, 118.0, 380.0, 44.0, 266.0, 210.0, 0.0, 434.0, 306.0, 436.0, 329.0], [358.0, 452.0, 462.0, 204.0, 126.0, 146.0, 252.0, 43.0, 369.0, 205.0, 444.0, 361.0, 314.0, 74.0, 350.0, 422.0, 454.0, 137.0, 488.0, 61.0, 462.0, 304.0, 325.0, 434.0, 0.0, 421.0, 178.0, 337.0], [354.0, 31.0, 56.0, 219.0, 351.0, 356.0, 170.0, 391.0, 233.0, 225.0, 56.0, 149.0, 107.0, 347.0, 236.0, 215.0, 195.0, 368.0, 425.0, 405.0, 351.0, 120.0, 435.0, 306.0, 421.0, 0.0, 299.0, 94.0], [400.0, 324.0, 325.0, 104.0, 59.0, 57.0, 142.0, 134.0, 191.0, 85.0, 303.0, 197.0, 197.0, 131.0, 338.0, 387.0, 404.0, 235.0, 526.0, 205.0, 476.0, 207.0, 414.0, 436.0, 178.0, 299.0, 0.0, 204.0], [347.0, 119.0, 125.0, 133.0, 257.0, 262.0, 86.0, 304.0, 167.0, 135.0, 106.0, 85.0, 34.0, 264.0, 239.0, 248.0, 245.0, 305.0, 443.0, 329.0, 374.0, 70.0, 410.0, 329.0, 337.0, 94.0, 204.0, 0.0]]


28

# **Main Code**

In [ ]:
results = []
for i in range(100):                                                              #################### change ####################### 10-1
    for spt in spoil_time :
        the_size = spt*200 ############## 200 ==>
        for prb in range(7):                                                
            start = process_time()
            donation_da = Read_donation(dons[prb]) 
            donation_data = np.array([np.array(xi) for xi in donation_da])
            location_data = Read_location(locs[prb])  
            total_distance = np.array([np.array(xi) for xi in location_data])
            backup_donation_data = dc(donation_data)
            roots = chrom_pool (donation_data,spt,the_size,total_distance)
            population_size = int(10 * np.ceil(np.log(len(roots))))             ############### 10 ===> 
            work_hours = 1500+spt
            min_test = dc(minTest(roots,total_distance))
            best_result = []
            best_sequence = []
            best_time = []
            for vehicle in range(vehicle_num[prb]):    

        # '''============================|initial population|=================================='''
                population,population_timing = initial_population(population_size,roots,min_test,work_hours,donation_data,spt,total_distance)
        # '''============================|initial fitness|=================================='''
                fitness = []
                for i in range(len(population)): 
                    fitness.append(multi_healthy_collected_plt(population[i],population_timing[i],donation_data,spt,total_distance))    
################### ok
                f_min, f_max = min(fitness),max(fitness)
                seed_min, seed_max, p_max = 0,10, population_size    ############ 1 ==> 0
                seed_num = []
                for i in range(len(population)):
                    seed_num.append(int(seed_min + (seed_max - seed_min)*(fitness[i]-f_min)/(f_max-f_min)))



################### تعداد سید ها چقد تولیدمثل کنن معلوم شد
                for iter in range(MaxIter):#############
                    for i in seed_num:
                        for j in range(seed_num[i]):
                            temp_pop,temp_time = dc(two_opt(population[i],total_distance,spt, iter, MaxIter, sigma_initial,sigma_final,roots))
                            population.append(temp_pop)
                            population_timing.append(temp_time)
                            fitness.append(multi_healthy_collected_plt(temp_pop,temp_time,donation_data,spt,total_distance)) 
################### سید های جدید به جمعیت اولیه افزوده شد
                    fitness,population,population_timing = dc(weak_del (fitness,population,population_timing,p_max))





################### ضعیف ها حذف شدند
                    for k in range(len(population)):
                        if len(population[k]) != len(population_timing[k]):
                            print('this is the mistake')
        # '''============================|result|=================================='''
                maxpos = fitness.index(max(fitness)) 
                best_result.append(fitness[maxpos])
                best_sequence.append(population[maxpos])
                best_time.append(population_timing[maxpos])
                donation_data = updating_donation_list (best_sequence[-1],best_time[-1],donation_data,total_distance)
            end = process_time()
            print(f'instance #{prb+1} CP is {sum(best_result)} in {end - start} seconds')
            results.append([prb+1,sum(best_result),end - start])
            # time_conflict(best_sequence,best_time,total_distance)
            # CP_check(donation_data,best_result)
            # counting_check (best_sequence,best_time,backup_donation_data,total_distance,spt,best_result)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


instance #1 CP is 111 in 63.186308565000004 seconds

instance #2 CP is 195 in 184.313966761 seconds

instance #3 CP is 223 in 150.60327243800003 seconds

instance #4 CP is 390 in 326.668444924 seconds

instance #5 CP is 321 in 313.0287328879999 seconds

instance #6 CP is 259 in 203.29180201999998 seconds

instance #7 CP is 329 in 328.309535159 seconds

instance #1 CP is 231 in 165.1743435530002 seconds

instance #2 CP is 369 in 267.31851367900003 seconds

instance #3 CP is 397 in 230.82869907100007 seconds

instance #4 CP is 539 in 382.80125399300005 seconds

instance #5 CP is 585 in 462.73263468899995 seconds

instance #6 CP is 573 in 607.059955408 seconds

instance #7 CP is 584 in 457.49282666999943 seconds

instance #1 CP is 418 in 191.31499689800057 seconds

instance #2 CP is 486 in 275.10718143099984 seconds

instance #3 CP is 507 in 279.40323456699934 seconds

instance #4 CP is 715 in 313.7432760949996 seconds

instance #5 CP is 756 in 355.09681768799965 seconds

instance #6 CP i